In [1]:
import pandas as pd
import numpy as np

In [2]:
#read data file
df = pd.read_csv('./cnp 3-yrs_web-scrap.csv')
df

,Unnamed: 0,P-ASP Date,Building / Address,Floor,Unit,GFA (ft??),SA (ft??),Leased / Sold,Price(HK$),Price/ft2(GFA),Price/ft2(SA),Source
0,3.0,30/4/2022,Yau Tong??,23,E,857,687,Leased,"20,000",@23,@29,CENTURY 21 HOME-LY PROPERTY AGENCY
1,4.0,30/4/2022,Yau Tong??,18,B,644,517,Leased,"15,000",@23,@29,CENTURY 21 HOME-LY PROPERTY AGENCY
2,5.0,30/4/2022,Yau Tong??,25,A,939,748,Leased,"21,500",@23,@29,CENTURY 21 HOME-LY PROPERTY AGENCY
3,6.0,30/4/2022,Yau Tong??,22,H,939,748,Leased,"20,500",@22,@27,CENTURY 21 HOME-LY PROPERTY AGENCY
4,7.0,30/4/2022,Yau Tong??,7,C,664,522,Leased,"16,500",@25,@32,CENTURY 21 HOME-LY PROPERTY AGENCY
...,...,...,...,...,...,...,...,...,...,...,...,...
5068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#drop unnecessary columns and drop NA rows
df = df.drop(columns=['Unnamed: 0','GFA (ft??)','Leased / Sold','Price/ft2(GFA)','Source','Unit'], axis=1)
df = df.dropna(axis=0)
df = df.rename(columns={"P-ASP Date": "Date", "SA (ft??)": "SA (ft2)"})

In [4]:
#Cleaning date to datetime format
df["NDate"] = pd.to_datetime(df["Date"],dayfirst=True,exact=False)
df = df.drop(columns=['Date'], axis=1)
df = df.rename(columns={"NDate": "Date"})

In [5]:
#add a district column
df["District"] = df["Building / Address"].apply(lambda x: x[:x.find("?")])

In [6]:
import statistics

In [7]:
#Determine how to group the floors into 'High','Mid','Low' categories
f = [int(i) for i in df["Floor"] if i.isdigit()]
l,m,h = (1,int(max(f)/3)),(int(max(f)/3)+1,int(max(f)*(2/3))),(int(max(f)*(2/3))+1,max(f))
print(l,m,h)

(1, 10) (11, 21) (22, 32)


In [8]:
def floor_change(x):
    if x.isdigit() and int(x) >= l[0] and int(x) <= l[1]:
        return "L"
    elif x.isdigit() and int(x) >= m[0] and int(x) <= m[1]:
        return "M"
    elif x.isdigit() and int(x) >= h[0] and int(x) <= h[1]:
        return "H"
    else:
        return x

In [9]:
df["Floor"] = df["Floor"].apply(lambda x: floor_change(x))

In [10]:
index_names = df[(df['Floor'] != "L") & (df['Floor'] != "M") & (df['Floor'] != "H") ].index
# drop these row indexes
# from dataFrame
df.drop(index_names, inplace = True)

In [11]:
#remove non-numeric str
df["SA (ft2)"] = df["SA (ft2)"].str.replace(r'\D', '')
df["Price(HK$)"] = df["Price(HK$)"].str.replace(r'\D', '')
#change type to int
df["SA (ft2)"] = df["SA (ft2)"].astype(int)
df["Price(HK$)"] = df["Price(HK$)"].astype(int)
#create a new column Price/ft2(SA)
df["Price/ft2(SA)"] = round(df["Price(HK$)"]/df["SA (ft2)"],2)

C:\Users\HiuC Lau\AppData\Local\Temp\ipykernel_9948\273350754.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["SA (ft2)"] = df["SA (ft2)"].str.replace(r'\D', '')
C:\Users\HiuC Lau\AppData\Local\Temp\ipykernel_9948\273350754.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Price(HK$)"] = df["Price(HK$)"].str.replace(r'\D', '')


In [12]:
import requests
import json

#Find district from estate
headers ={"Accept": "application/json", "Accept-Language" : "en"}
url = "https://www.als.ogcio.gov.hk/lookup"

districts =[]

for i in df['District']:
    address = i
    params = {"q":address}
    res = requests.get(url, params=params, headers=headers)
    d = json.loads(res.text)
    try: dist = d['SuggestedAddress'][0]['Address']['PremisesAddress']['EngPremisesAddress']['EngDistrict']['DcDistrict']
    except: dist = 'None'
    districts.append(dist)

In [13]:
df['District1'] = districts
df['District1'] = df['District1'].str.replace(' DISTRICT','')

In [14]:
df

,Building / Address,Floor,SA (ft2),Price(HK$),Price/ft2(SA),Date,District,District1
0,Yau Tong??,H,687,20000,29.11,2022-04-30,Yau Tong,KWUN TONG
1,Yau Tong??,M,517,15000,29.01,2022-04-30,Yau Tong,KWUN TONG
2,Yau Tong??,H,748,21500,28.74,2022-04-30,Yau Tong,KWUN TONG
3,Yau Tong??,H,748,20500,27.41,2022-04-30,Yau Tong,KWUN TONG
4,Yau Tong??,L,522,16500,31.61,2022-04-30,Yau Tong,KWUN TONG
...,...,...,...,...,...,...,...,...
549,Yau Tong??LAGUNA CITY PH 01 BLK 08 ...,M,687,22500,32.75,2021-10-06,Yau Tong,KWUN TONG
550,Yau Tong??LAGUNA CITY PH 03 BLK 37 ...,M,517,16300,31.53,2021-10-06,Yau Tong,KWUN TONG
552,Tseung Kwan O??METRO CITY PH 01 TWR 02,H,364,15000,41.21,2021-10-05,Tseung Kwan O,SAI KUNG
553,Tseung Kwan O??BEVERLY GDN PH 02 BLK 09 (PSPS),L,484,14500,29.96,2021-10-05,Tseung Kwan O,SAI KUNG


In [15]:
df = df.drop(columns=['Building / Address'], axis=1)
df

,Floor,SA (ft2),Price(HK$),Price/ft2(SA),Date,District,District1
0,H,687,20000,29.11,2022-04-30,Yau Tong,KWUN TONG
1,M,517,15000,29.01,2022-04-30,Yau Tong,KWUN TONG
2,H,748,21500,28.74,2022-04-30,Yau Tong,KWUN TONG
3,H,748,20500,27.41,2022-04-30,Yau Tong,KWUN TONG
4,L,522,16500,31.61,2022-04-30,Yau Tong,KWUN TONG
...,...,...,...,...,...,...,...
549,M,687,22500,32.75,2021-10-06,Yau Tong,KWUN TONG
550,M,517,16300,31.53,2021-10-06,Yau Tong,KWUN TONG
552,H,364,15000,41.21,2021-10-05,Tseung Kwan O,SAI KUNG
553,L,484,14500,29.96,2021-10-05,Tseung Kwan O,SAI KUNG


In [16]:
df.to_csv("cleaned_cnp 3-yrs_web-scrap (+18dis).csv", index= False)